In [ ]:
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets,transforms
import torch.optim as optim
import torch
y=[]
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(degrees=10),
    transforms.RandomCrop(size=32,padding=4),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616))])
test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5),(0.5))])
train_dataset=datasets.CIFAR10('./data',train=True,download=True,transform=train_transform)
test_dataset = datasets.CIFAR10('./data',train=False,download=True,transform=test_transform)
train_loader = DataLoader(dataset=train_dataset,batch_size=128,shuffle=True)
test_loader = DataLoader(dataset=test_dataset,batch_size=1000,shuffle=False)
model = nn.Sequential(
    nn.Conv2d(3,16,3,1,1),
    nn.BatchNorm2d(16),
    nn.ReLU(),
    nn.MaxPool2d(2,2),
    nn.Conv2d(16,32,3,1,1),
    nn.BatchNorm2d(32),
    nn.ReLU(),
    nn.MaxPool2d(2,2),
    nn.Conv2d(32, 64, 3, 1, 1),
    nn.BatchNorm2d(64),
    nn.ReLU(),
    nn.MaxPool2d(2,2),
    nn.Flatten(),
    nn.Linear(64*4*4,128),
    nn.ReLU(),
    nn.Dropout(0.15),
    nn.Linear(128,10)
).to(device)
optimizer = optim.Adam(model.parameters(),lr=0.001)
loss_func = nn.CrossEntropyLoss()
EPOCHS = 10
for epoch in range(EPOCHS):
    model.train()
    total_loss = 0
    for images,labels in train_loader:
        images,labels=images.to(device),labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = loss_func(outputs,labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    avg_loss = total_loss / len(train_loader)
    model.eval()
    total,correct=0,0
    with torch.no_grad():
        for images,labels in test_loader:
             images,labels=images.to(device),labels.to(device)
             outputs = model(images)
             pred = torch.argmax(outputs,dim=1)
             correct += (pred == labels).sum().item()
             total += labels.size(0)
        accuracy = correct / total * 100
        print(f"EPOCH: {epoch+1} - avg loss: {avg_loss}, accuracy: {accuracy}%")

Using device: cpu
EPOCH: 1 - avg loss: 1.5760466061589662, accuracy: 32.57%
EPOCH: 2 - avg loss: 1.274059700843928, accuracy: 41.05%
EPOCH: 3 - avg loss: 1.1708446024628856, accuracy: 43.5%
EPOCH: 4 - avg loss: 1.089306948130088, accuracy: 47.08%
EPOCH: 5 - avg loss: 1.0253001578018794, accuracy: 57.120000000000005%
EPOCH: 6 - avg loss: 0.9903338365542614, accuracy: 48.64%
EPOCH: 7 - avg loss: 0.9545611623303055, accuracy: 51.31%
EPOCH: 8 - avg loss: 0.9309909297987018, accuracy: 52.959999999999994%
EPOCH: 9 - avg loss: 0.9013011524134584, accuracy: 53.080000000000005%
EPOCH: 10 - avg loss: 0.8866805261968042, accuracy: 61.029999999999994%
